In [1]:
!pip install -e ..

In [2]:
import gdown
import shutil
from astrohack.panel import panel
from astrohack._utils._fits_io import _aips_holog_to_xds
import os
from dask.distributed import Client, LocalCluster

In [3]:
# Download VLA Holography with known panel deviations
datafolder = "./paneldata/"
os.makedirs(name=datafolder, exist_ok=True)
panelzip = datafolder+"panel.zip"
if not os.path.exists(panelzip):
    url = "https://drive.google.com/u/1/uc?id=10fXyut_UHPUjIuaaEy6-m6wcycZHit2v&export=download"
    gdown.download(url, panelzip)
shutil.unpack_archive(filename=panelzip, extract_dir=datafolder)

In [4]:
# Create .zarr data set from AIPS data
holog_image = datafolder+'input.zarr'
input_data_xds = _aips_holog_to_xds(datafolder+'amp.fits', datafolder+'dev.fits')
input_data_xds.to_zarr(holog_image, mode='w')

In [5]:
%%time
# Execute Panel on a single VLA dataset
panel(holog_image, 'vla_data',
      panel_kind='corotatedparaboloid',
      aipsdata=True, 
      telescope='vla', 
      basename='antenna25', 
      save_mask=False, 
      save_phase=False, 
      unit='miliinches', 
      cutoff=0.2,
      panel_margins=0.2)

In [6]:
# Initialize local cluster
#asdrubal
try:
    client = Client('http://127.0.0.1:8787')

except:
    cluster = LocalCluster(
        n_workers=4,
        threads_per_worker=1,
        memory_limit='16GB'
    )

    client = Client(cluster)

In [7]:
%%time
holog_image = 'hack_file'
panel_output = 'hack_file'
panel(holog_image, panel_output,
      panel_kind='corotated_lst_sq',
      aipsdata=False, 
      telescope=None, 
      save_mask=False, 
      save_phase=False, 
      unit='mm', cutoff=0.2)